# Lesson 3 - Example Model

In [3]:
import os
from ansys.mapdl.core import launch_mapdl
from path_tools import *

# Define project directory and TMP folder
project_dir = os.getcwd()
tmp_dir = os.path.join(project_dir, "tmp")

# Create tmp directory if it doesn't exist
os.makedirs(tmp_dir, exist_ok=True)

# Path to the ANSYS executable
ansys_path = "C:\\Program Files\\ANSYS Inc\\v242\\ansys\\bin\\winx64\\ANSYS242.exe"

# Convert the Windows path to WSL path
ansys_path = windows_to_wsl_path(ansys_path)

# Job name
job_name = "strain_dataset"
print("Run New MAPDL instance")

Run New MAPDL instance


In [4]:
mapdl = launch_mapdl(run_location=tmp_dir, 
                     remove_temp_dir_on_exit = True, 
                     cleanup_on_exit = True,
                     jobname = job_name,
                     exec_file = ansys_path,
                     additional_switches = '-smp', 
                     nproc = 2)

/home/thanglx/miniconda3/envs/ansys/lib/python3.10/site-packages/ansys/tools/path/path.py:1002: UserWarning: No path found for mapdl in default locations.
  warnings.warn(f"No path found for {product} in default locations.")


In [5]:
Length_Inches = 4.5
Height_Inches = 2

sensors = { 'x1': 1,    'y1': .5,
            'x2': 2.25, 'y2': .5,
            'x3': 3.5,  'y3': .5,
            'x4': 1,    'y4': 1.5,
            'x5': 2.25, 'y5': 1.5,
            'x6': 3.5,  'y6': 1.5,
            'x7': .5,   'y7': 1,
            'x8': 4,    'y8': 1
          }

mapdl.clear()
mapdl.prep7()
mapdl.blc4(0, 0, Length_Inches, Height_Inches, 0)
mapdl.adele('all')

mapdl.lfillt(1,2,.2)
mapdl.lfillt(2,3,.2)
mapdl.lfillt(3,4,.2)
mapdl.lfillt(4,1,.2)

mapdl.numcmp('area')
mapdl.al('all')

for points in range(1, int((len(sensors)/2)+1)):
    mapdl.hptcreate('area',1,'','coord',sensors['x'+str(points)], sensors['y'+str(points)])


In [6]:
mapdl.et(1,181)
    
mapdl.mp('ex',1,1e7)
mapdl.sectype(1,'shell')
mapdl.secdata(.1,1)

mapdl.mshape(1,'2d')
mapdl.aesize('all',.05)
mapdl.amesh('all')

mapdl.allsel()
mapdl.nsll('u',1)
mapdl.nslk('u')

node_count = int(mapdl.get_value('node',0,'count'))
print(node_count)

3979


In [7]:
nodes = mapdl.mesh.nodes
f = open('pointset.csv', 'w')
for node in nodes:
    f.write(str(node[0]) + ' , ' + str(node[1]) + '\n')
f.close()

In [8]:
mapdl.exit()